In [1]:
import numpy as np
CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

In [2]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\bikas_nfppdsb\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\bikas_nfppdsb\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
from nltk.corpus import treebank
sentences = treebank.tagged_sents(tagset='universal')

In [5]:
import random
print(random.choice(sentences))

[('Other', 'ADJ'), ('paper', 'NOUN'), ('and', 'CONJ'), ('forest-products', 'NOUN'), ('stocks', 'NOUN'), ('closed', 'VERB'), ('*-1', 'X'), ('mixed', 'VERB'), ('.', '.')]


### Find all tags

In [6]:
tags = set([
    tag for sentence in treebank.tagged_sents() for _, tag in sentence
])
print('nb_tags: %sntags: %s' % (len(tags), tags))
print(len(tags))

nb_tags: 46ntags: {'VBN', 'WP', 'NNP', 'CD', 'NNS', 'RBR', 'CC', 'RB', 'NNPS', 'WP$', ',', '``', 'VBP', '-RRB-', 'TO', 'RP', 'PRP$', 'FW', 'VBG', 'JJR', '$', 'LS', '-LRB-', 'RBS', 'MD', 'NN', 'POS', 'VB', '.', 'IN', 'UH', '#', 'PRP', 'WRB', 'SYM', 'JJS', "''", ':', 'VBD', 'VBZ', 'JJ', 'PDT', 'EX', '-NONE-', 'WDT', 'DT'}
46


## Data processing

In [7]:
train_test_cutoff = int(.80 * len(sentences)) 
training_sentences = sentences[:train_test_cutoff]
testing_sentences = sentences[train_test_cutoff:]

# 25% of remaining data in to validation and rest in train
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

## Feature Engineering

In [8]:
def add_basic_features(sentence_terms, index):
    """ Compute some very basic word features.
        :param sentence_terms: [w1, w2, ...] 
        :type sentence_terms: list
        :param index: the index of the word 
        :type index: int
        :return: dict containing features
        :rtype: dict
    """
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

## Helper Function

In [9]:
def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term.
    :param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.
    :param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []
    
    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(pos_tags), index))
            y.append(class_)
            
    return X, y

### Prepare the final dataset

Here the type of X gonna be list of dictionary <br/>
        and the  type of y will be list of class

In [10]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [11]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

print(type(X_train))
print(type(X_train[0]))
print(X_train[0])

print()
print(type(y_train))
print(type(y_train[0]))

61107
20039
19530
<class 'list'>
<class 'dict'>
{'nb_terms': 28, 'term': 'Mrs.', 'is_first': True, 'is_last': False, 'is_capitalized': True, 'is_all_caps': False, 'is_all_lower': False, 'prefix-1': 'M', 'prefix-2': 'Mr', 'prefix-3': 'Mrs', 'suffix-1': '.', 'suffix-2': 's.', 'suffix-3': 'rs.', 'prev_word': '', 'next_word': 'Ward'}

<class 'list'>
<class 'str'>


## Feature encoding
Dict can not be feeded into neural network. So, we need to encode it into vector.

In [12]:
from sklearn.feature_extraction import DictVectorizer

# Fit our DictVectorizer with our set of features
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)

# Convert dict features to vectors
X_train = dict_vectorizer.transform(X_train)
# X_test = dict_vectorizer.transform(X_test)
# X_val = dict_vectorizer.transform(X_val)

In [14]:
print(type(X_train))
print(X_train.shape)
print(X_train[0])

<class 'numpy.ndarray'>
(61107, 44232)
[0. 0. 1. ... 0. 0. 0.]


### Encoding and one-hot encoding y

In [15]:
from sklearn.preprocessing import LabelEncoder

# Fit LabelEncoder with our list of classes
label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_test + y_val)

# Encode class values as integers
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [18]:
print(y_train.shape)
y_train

(61107,)


array([ 6,  6, 10, ...,  6,  6,  0], dtype=int64)

In [19]:
# Convert integers to dummy variables (one hot encoded)
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

Using TensorFlow backend.


In [20]:
y_train.shape

(61107, 12)

## Building the model

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

def build_model(input_dim, hidden_neurons, output_dim):
    """
    Construct, compile and return a Keras model which will be used to fit/predict
    """
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('relu'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('relu'),
        Dropout(0.2),
        Dense(output_dim, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier

model_params = {
    'build_fn': build_model,
    'input_dim': X_train.shape[1],
    'hidden_neurons': 512,
    'output_dim': y_train.shape[1],
    'epochs': 5,
    'batch_size': 256,
    'verbose': 1,
    # 'validation_data': (X_val, y_val),
    'shuffle': True
}

clf = KerasClassifier(**model_params)

### Train the model

In [ ]:
hist = clf.fit(X_train, y_train)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
61107/61107 [==============================] - 354s 6ms/step - loss: 0.3907 - acc: 0.8860
Epoch 2/5
61107/61107 [==============================] - 354s 6ms/step - loss: 0.0707 - acc: 0.9773
Epoch 3/5
31232/61107 [==============>...............] - ETA: 3:13 - loss: 0.0364 - acc: 0.9884